# Connect to MySQL

In [8]:
import json
import pymysql
from pymysql import connect

In [13]:
try:
    conn = connect(
        host='localhost',
        port=3306,
        user='root',
        password='XJWxjw1998',
        db='my_twitter_db',
        charset = 'utf8mb4'
    )
    
    cursor = conn.cursor()
except Exception as e:
    print(e)
else:
    print('Connect Success:%s' % cursor)

Connect Success:<pymysql.cursors.Cursor object at 0x7fe3e0565dc0>


In [14]:
d = "SHOW TABLES"
cursor.execute(d)
cursor.fetchall()

(('user_info',),)

In [15]:
cursor.execute("SHOW columns FROM user_info")
col = [] #create the col for later output format
for i in cursor.fetchall():
    print(i[0])
    col.append(i[0])
    #print('col:',col)

user_id
name
screen_name
location
url
protected
followers_count
friends_count
listed_count
favourites_count
created_at
statuses_count
following
follow_request_sent


In [12]:
### Close connection for MySQL
cursor.close()
conn.close()

# Connet to MongoDB

In [16]:
import sys
import json
import pymongo
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

client = pymongo.MongoClient('mongodb://localhost:27017')
database = client['twitter1']
collection = database['tweets']
twitter = client.twitter

# Seach App

In [17]:
##### function for 'tweet content' search #####

def find_tweet_info(input_matching_text):
    cursor = twitter1.tweets.find({"text": {'$regex': input_matching_text}}, projection = projection, sort = sort, limit =5)
    
    for doc in cursor:
            try:
                print('\nTweet: ',doc['retweeted_status']['text'])
                print('retweet count: ', doc['retweeted_status']['retweet_count'])
                print('user id: ', doc['user']['id'])

            except KeyError:
                print("No result")
            

In [18]:
def find_hashtag_info(input_matching_hashtag):
    cursor = twitter1.tweets.find({"entities.hashtags.text": {'$regex': input_matching_hashtag}}, 
                                 projection = projection, sort = sort, limit = 5)
    
    for doc1 in cursor:
        print('\nTweets: ',doc1['text'])
        print('user id: ', doc1['user']['id'])

In [19]:
import time

In [20]:
while True:
    question = input('*****Options***** \n- user id \n- user screen name \n- tweets \n- hashtag \n- rank \n- quit \nWhich option you want to choose? ')

                                #######################
                                ### MySQL Database ####
                                #######################
    ###########################            
    ######### user id #########
    ###########################
    if question == 'user id':
        
    
        print('\nSearch by user id...')
        user_id = int(input('\nPlease enter the user id:'))
        print('\n')
        print('RESULT:')

        print('\n[User Information]')
        
        ## TIME
        start = time.time()
        
        #### user info #### MySQL
        search_user_id = "SELECT * from user_info WHERE user_id = %d;" % (user_id)#1000083595548127232
        cursor.execute(search_user_id)
        result = cursor.fetchall()
#         print(result) # ()
#         print(len(result))
        #print(result[0])
        
        if len(result)==0:
            print('There is no such user.')
            
        else:
            # len(result[0]) =14
            for i in range(14):
                print(col[i], ': ',result[0][i])

            #### user tweet #### mongoDB
            print('\n[Tweet]')
            q = collection.find({"user.id": user_id})

            for i in q:
                print('tweet:',i['text'])


            ## TIME
            end = time.time()
            print('\nRuning Time: ',end-start)
            print('\n')
                
        
        #### user RE-tweet #### mongoDB
#         print('\n')
#         print('[RE-Tweet]')
#         q = collection.find({:})
        
#         for i in q:
#             print('RE-tweet:',i['text'])
        
       
        
    #############################
    ######### user name #########
    #############################
    elif question == 'user screen name':

        print('\nSearch by user screen name...')
        screen_name = input('\nPlease enter the user screen name:')
        print('\n')
        print('RESULT:')

        print('\n[User Information]')
        
        start = time.time()## TIME
        search_user_screen_name = "SELECT * from user_info WHERE screen_name = '%s';" % (screen_name)#marianap07
        cursor.execute(search_user_screen_name)
        result = cursor.fetchall()
        #print(result) # ()
        #print(result[0])
        
        if len(result)==0:
            print('There is no such user.')
            
        else:
            # len(result[0]) =14
            for i in range(14):
                print(col[i], ': ',result[0][i])

            #### user tweet #### mongoDB
            print('\n[Tweet]')
            q = collection.find({"user.screen_name": screen_name})

            for i in q:
                print('tweet:',i['text'])


            ## TIME
            end = time.time()
            print('\nRuning Time: ',end-start)
            print('\n')
        
            
                                #######################
                                ## Mongodb Database ###
                                #######################
    ##########################                
    ######### tweets #########
    ##########################
    elif question == 'tweets':
        print('Searching by tweets...')
        
        projection = {}
        projection["user.id"] = 1.0
        projection["retweeted_status.text"] = 1.0
        projection["retweeted_status.retweet_count"] = 1.0
        sort = [ (u"retweeted_status.retweet_count", -1) ]

        textMatching = input('please enter content:')
        print('\n')
        print('RESULT:')
        
        start = time.time()
        input_matching_text = textMatching
        find_tweet_info(input_matching_text)
        
        ## TIME
        end = time.time()
        print('\nRuning Time: ',end-start)
        print('\n')


        # MySQL user information
        option = input('\nWould you like to search for the user information of one tweet above? \n- yes \n- no \n')
        if option == "yes":
            user_id = int(input('\nPlease paste the user id of the tweet above:'))
            print('\n')
#             print('RESULT:')
            print('[User Information]')
            
            search_user_id = "SELECT * from user_info WHERE user_id = %d;" % (user_id)
            cursor.execute(search_user_id)
            result = cursor.fetchall()
            for i in range(14):# len(result[0]) =14
                print(col[i], ': ',result[0][i])

        elif option == "no":
            print('Good Bye~')
    
    ###########################
    ######### hashtag #########
    ###########################
    elif question == 'hashtag':
        print('Searching by hashtag...')
    
        projection = {}
        projection["user.id"] = 1.0
        projection["text"] = 1.0
        projection["entities.hashtags"] = 1.0
        sort = [ (u"created_at", -1) ]

        #input
        textMatching1 = input('please enter hashtag:')
        start = time.time()
        
        print('\n')
        print('RESULT:')
        input_matching_hashtag = textMatching1
        find_hashtag_info(input_matching_hashtag)
        ## TIME
        end = time.time()
        print('\nRuning Time: ',end-start)
        print('\n')

            # MySQL user information
        option = input('\nWould you like to search for user information of one tweet above? \n- yes \n- no \n')
        if option == "yes":
            user_id = int(input('\nPlease paste the user id of the tweet above:'))
            print('\n')
#             print('RESULT:')
#             print('\n')
            print('[User Information]')
            search_user_id = "SELECT * from user_info WHERE user_id = %d;" % (user_id)
            cursor.execute(search_user_id)
            result = cursor.fetchall()

            # len(result[0]) =14
            for i in range(14):
                print(col[i], ': ',result[0][i])
        elif option == "no":
            print('Good Bye')

    ########################
    ######### Rank #########
    ########################
    elif question == 'rank':
        print('Searching by rank...')
    
        option = input('What type rank would you like to see: \n- mention \n- hashtag \nWhich option you want to choose? ')
        
        print('\n')
        print('RESULT:')
        if option == 'mention':
            start = time.time()
            for doc in twitter.usermentions.find({}).sort('value', -1).limit(10):
                print(doc)

            ## TIME
            end = time.time()
            print('\nRuning Time: ',end-start)
            print('\n')

        elif option == 'hashtag':
            start = time.time()
            for doc in twitter.hashtags.find({}).sort('value', -1).limit(10):
                print(doc)

            ## TIME
            end = time.time()
            print('\nRuning Time: ',end-start)
            print('\n')
    
    
    ######### Quit #########
    elif question == 'quit':
        print('Good Bye~')
        break
    
    ######### input error check ######### 
    else:
        print('ERROR! Please enter the correct option.')
        print('\n')
    

*****Options***** 
- user id 
- user screen name 
- tweets 
- hashtag 
- rank 
- quit 
Which option you want to choose? user id

Search by user id...

Please enter the user id:3130596629


RESULT:

[User Information]
user_id :  3130596629
name :  Arkansas Boy
screen_name :  ArkansasWatch
location :  #Ark #LittleRock
url :  None
protected :  0
followers_count :  276
friends_count :  867
listed_count :  16
favourites_count :  1128
created_at :  Tue Mar 31 08:40:52 +0000 2015
statuses_count :  4410
following :  None
follow_request_sent :  None

[Tweet]
tweet: We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump #ashamed @ArpxFeed https://t.co/rb2E0FTm3V
tweet: We have 1/4 of all deaths from #Corona - thanks to our 3rd world response by @realDonaldTrump  #ashamed 
@Arpx… https://t.co/3sP4AIJNJc

Runing Time:  2.7382290363311768


*****Options***** 
- user id 
- user screen name 
- tweets 
- hashtag 
- rank 
- quit 
Which option you want to choose? u

## Redis Connection


In [ ]:
import time #For handling timing executionimport redis
import datetime # For handling datetime objects
from datetime import timedelta # For handling redis cache time to live 
import time #For handling timing execution

In [ ]:
redis_client = redis.Redis(host='localhost', port='6379', decode_responses=True)

In [ ]:
# my_doc = twitter.hashtags.find({}).sort('value', -1).limit(5)  ## twitter.hashtags.find({}).sort('value', -1)
            
#          # "_id" : "COVID19", "value" : 102
# for doc in my_doc:
#     print('Popular Hashtags:', doc['_id'],'Count:',doc['value'])

In [ ]:
def hashtag_rank_search_helper(redis_key: str) -> str:
        
        summary = ""
        start_time = time.time()
        
        if redis_client.exists(redis_key) > 0 and redis_client.ttl(redis_key) > 0:

            msg1 = "Found in redis cache..."
            ## get from redis cache ##
            summary += redis_client.get(redis_key)
            
            end_time = time.time()
            elapsed_time = end_time - start_time

            elapsed_time_ms = str(round(elapsed_time * 1000)) + 'ms'
            msg2 = "   *** Summary generation time:" + elapsed_time_ms
        
        else:
            msg1 = "Not found in redis cache... Pulling data from DB and updating cache"
            ## query from mongoDB ##
            my_doc = twitter.hashtags.find({}).sort('value', -1).limit(5)
            
            for doc in my_doc:
                print(doc)
                tag = doc['_id']
                #print(tag)
                count = doc['value']
                #print(count)
                summary += """  ## Popular Hashtag : {} , Count: {} ## """.format(doc['_id'], doc['value'])
            
            end_time = time.time()
            elapsed_time = end_time - start_time

            elapsed_time_ms = str(round(elapsed_time * 1000)) + 'ms'

            msg2 = "   *** Summary generation time:" + elapsed_time_ms

            
            redis_client.setex(redis_key, time=timedelta(minutes=15), value=summary)
            
        return msg1 + summary + msg2

In [25]:
redis_key = """{}:{}""".format(1, "hashtags")
hashtag_rank_search_helper(redis_key)

'Found in redis cache...  ## Popular Hashtag : Corona , Count: 599.0 ##   ## Popular Hashtag : corona , Count: 248.0 ##   ## Popular Hashtag : HappyEaster2020 , Count: 129.0 ##   ## Popular Hashtag : COVID19InTurkeysPrisons , Count: 109.0 ##   ## Popular Hashtag : COVID19 , Count: 102.0 ##    *** Summary generation time:1ms'

In [25]:
redis_key = """{}:{}""".format(1, "hashtags")
hashtag_rank_search_helper(redis_key)

'Found in redis cache...  ## Popular Hashtag : Corona , Count: 599.0 ##   ## Popular Hashtag : corona , Count: 248.0 ##   ## Popular Hashtag : HappyEaster2020 , Count: 129.0 ##   ## Popular Hashtag : COVID19InTurkeysPrisons , Count: 109.0 ##   ## Popular Hashtag : COVID19 , Count: 102.0 ##    *** Summary generation time:1ms'